In [ ]:
# 📦 Import required modules
import scanpy as sc
from scripts.priorcci_preprocess import input_data_preprocess
from scripts.data_loader import load_data
from scripts.train import train_and_save_model
from scripts.evaluate import evaluate_saved_models
from scripts.utils import f1_m, precision_m, recall_m
from scripts.visualize import visualize_final_model_results
from scripts.gradcam_module import run_gradcam_analysis
from scripts.merge_gradcam_results import merge_gradcam_results
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split


In [ ]:
# 🔬 Step 1: Preprocess input data for CNN
adata = sc.read('DB/CCA_Lung_toy.h5ad')
input_data_preprocess(
    adata,
    celltype_col='cell_type_major',
    output_folder='cnn_input_data',
    n_repeat=1000
)


In [ ]:
# 📂 Step 2: Load .npz samples and prepare training labels
data, labels, num_classes, num_lrpair = load_data(path='cnn_input_data/')


In [ ]:
# 🧠 Step 3: Train and save CNN models
train_and_save_model(data, labels, num_classes, num_lrpair)


In [ ]:
# 📊 Step 4: Evaluate all trained models
evaluate_saved_models(data, labels, num_classes)


In [ ]:
# 📈 Step 5: Visualize final model (v10)
model_path = 'cnn_model/data_cnn-model_v010.h5'
model = load_model(model_path, custom_objects={
    'f1_m': f1_m,
    'precision_m': precision_m,
    'recall_m': recall_m
})

_, test_data, _, test_labels = train_test_split(data, labels, test_size=0.2, random_state=51)
visualize_final_model_results(model, test_data, test_labels)


In [ ]:
# 🔍 Step 6: Run GradCAM++ analysis
import os
import re

def extract_number(filename):
    return int(re.search(r'\d+', filename).group())

class_files = sorted([f for f in os.listdir('cnn_input_data/') if f.endswith('.npz')], key=extract_number)

custom_objs = {'f1_m': f1_m, 'precision_m': precision_m, 'recall_m': recall_m}

run_gradcam_analysis(
    data=data,
    gene_list_csv='DB/filtered_CCIdb.csv',
    model_dir='cnn_model/',
    save_dir='gcam_res/',
    class_files=class_files,
    custom_objects=custom_objs
)


In [ ]:
# 📊 Step 7: Merge GradCAM++ results
merge_gradcam_results(
    path='gcam_res/',
    save_path='CCI_res/'
)
